## SVM / MNIST

### 패키지

In [2]:
import numpy as np
import random

from keras.datasets import mnist

from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

import json
import time

Using TensorFlow backend.


In [3]:
# Constants & Functions
model_name = 'mnist_svm'

def get_hot_idx(arr):
    return arr.index(max(arr))

def save_to_json_file(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)
    print(filename + ' 저장완료')
    
def get_round_array(array, decimal):
    return [round(e, decimal) for e in array]

current_milli_time = lambda: int(round(time.time() * 1000))

### 데이터

In [27]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Change from matrix to array --> dimension 28x28 to array of dimention 784
x_train = x_train[:5000].reshape(5000, 784)
x_test = x_test[:1000].reshape(1000, 784)

# Change to float datatype
x_train = x_train.astype('float32')[:5000]
x_test = x_test.astype('float32')[:1000]

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#I mod2 the label sets to have a result of 1 or 0 , for odds and evens respectively
y_train = y_train[:5000]
y_test = y_test[:1000]

5000 train samples
10000 test samples


### PCA

In [36]:
n_components = 16
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(x_train)

x_train_pca = pca.transform(x_train)

### 모델 구축

In [37]:
svm = SVC()

### 모델 학습 및 그리드 서치

In [38]:
print("grid search")
param_grid = { "C" : [0.1]
              , "gamma" : [0.1]}
grid = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1, verbose=1)

grid search


In [39]:
print("grid.fit")
grid = grid.fit(x_train_pca, y_train)

grid.fit
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    2.4s finished


In [40]:
#Now we train the best estimator in the full dataset
print("training svm")
best_svm = grid.best_estimator_
best_svm.fit(x_train , y_train)
# print(best_svm)
print("svm done")

training svm
svm done


### Test

In [41]:
print("Testing")
print("score: ", best_svm.score(x_test, y_test,))

Testing
score:  0.1135
